## 강의 1: 시계열 데이터 분석 EDA (Bike Sharing Demand)

- **목표**: `train.csv` 기반 시계열 구조 이해, 전처리, 시각화, 통찰 도출
- **데이터**: 시간 단위 대여량(`count`), 기상/달력 특징 포함
- **아웃풋**: EDA 체크리스트, 모형화에 쓰일 파생변수 후보, 데이터 스플릿 기준

### 커리큘럼(30분)
1. 데이터 개요와 로드 (3분)
2. 시계열 기본 전처리: 타입/결측/기상 스케일 점검 (5분)
3. 시계열 탐색: 추세/계절성/요일/시간대/날씨 (10분)
4. 상관/라그 특징 아이디어 (6분)
5. 베이스라인 분할 전략 및 리스크 (6분)



In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

DATA_PATH = os.path.join(os.getcwd(), 'bike-sharing-demand', 'train.csv')
print(f"로드 경로: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
print(df.head())
print(df.dtypes)

# datetime 파싱
df['datetime'] = pd.to_datetime(df['datetime'])

# 기본 통계
display(df.describe(include='all'))

# 결측치 점검
missing = df.isna().sum().sort_values(ascending=False)
print("결측치 요약:\n", missing[missing > 0])

# 파생 시간 변수
for col, func in {
    'year': lambda s: s.dt.year,
    'month': lambda s: s.dt.month,
    'day': lambda s: s.dt.day,
    'hour': lambda s: s.dt.hour,
    'weekday': lambda s: s.dt.weekday,
    'week': lambda s: s.dt.isocalendar().week.astype(int),
}.items():
    df[col] = func(df['datetime'])

# 시각화 샘플 크기 제한 (속도용)
df_sample = df.sample(n=min(2000, len(df)), random_state=42).sort_values('datetime')

fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.ravel()

# 시간대별 분포
sns.boxplot(x='hour', y='count', data=df_sample, ax=axes[0])
axes[0].set_title('시간대별 수요 분포')

# 요일별 분포
sns.boxplot(x='weekday', y='count', data=df_sample, ax=axes[1])
axes[1].set_title('요일별 수요 분포 (월=0)')

# 월별 평균 추세
sns.lineplot(x='month', y='count', data=df, estimator='mean', errorbar=None, ax=axes[2])
axes[2].set_title('월별 평균 수요')

# 날씨 영향
sns.scatterplot(x='temp', y='count', hue='weather', data=df_sample, ax=axes[3], palette='Set2')
axes[3].set_title('온도 vs 수요 (날씨 그룹)')

# 습도 영향
sns.scatterplot(x='humidity', y='count', hue='season', data=df_sample, ax=axes[4], palette='deep')
axes[4].set_title('습도 vs 수요 (계절)')

# 연도별 추세
sns.lineplot(x='datetime', y='count', data=df, ax=axes[5])
axes[5].set_title('시간 추세')
plt.tight_layout()
plt.show()

# 상관행렬
num_cols = ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']
plt.figure(figsize=(10, 7))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='crest', vmin=-1, vmax=1)
plt.title('수치 변수 상관관계')
plt.show()



In [ ]:
# 라그/롤링 특징 아이디어
# 모델링 강의에서 사용할 수 있도록 베이스라인 피처를 미리 계산

lag_hours = [1, 2, 3, 6, 12, 24]
for h in lag_hours:
    df[f'count_lag_{h}'] = df['count'].shift(h)

# 24시간 이동평균/표준편차
window = 24
df['count_rollmean_24h'] = df['count'].rolling(window).mean()
df['count_rollstd_24h'] = df['count'].rolling(window).std()

print(df[[c for c in df.columns if 'count_' in c]].head(30))

# 학습/검증 분할 기준 제안: 시간 기반(2012-10을 기준으로 split)
split_time = pd.Timestamp('2012-10-01')
train_idx = df['datetime'] < split_time
valid_idx = df['datetime'] >= split_time

print('Train 기간:', df.loc[train_idx, 'datetime'].min(), '→', df.loc[train_idx, 'datetime'].max())
print('Valid 기간:', df.loc[valid_idx, 'datetime'].min(), '→', df.loc[valid_idx, 'datetime'].max())

# 저장 (모델링 노트북에서 재사용)
eda_out_path = os.path.join(os.getcwd(), 'bike-sharing-demand', 'train_eda_features.parquet')
df.to_parquet(eda_out_path, index=False)
print('저장 완료:', eda_out_path)

